# Exercício 2.10 - Orientação a Objetos (OO)

Solução completa com testes para as classes solicitadas: `EventoABC`, `DataHora`, `EventoUnico` e `EventoRecorrente`.

In [ ]:

from abc import ABC, abstractmethod
from datetime import datetime, timedelta

class EventoABC(ABC):
    def __init__(self, titulo: str, descricao: str):
        # atributos "privados" por convenção
        self._titulo = titulo
        self._descricao = descricao

    @abstractmethod
    def __str__(self):
        pass

    @abstractmethod
    def isConcluido(self, *args, **kwargs):
        pass


## Classe DataHora

- Atributo privado `_data_hora` do tipo `datetime`.
- Atributo de classe `FORMAT` = '%d/%m/%Y, %H:%M'.
- `data_hora` como property (getter retorna string formatada; setter recebe string e converte).

In [ ]:

class DataHora:
    FORMAT = '%d/%m/%Y, %H:%M'

    def __init__(self):
        self._data_hora = None  # iniciado como None

    @property
    def data_hora(self):
        # retorna string formatada ou None
        if self._data_hora is None:
            return None
        return self._data_hora.strftime(self.FORMAT)

    @data_hora.setter
    def data_hora(self, valor_str):
        # valor_str deve ser string no formato especificado
        try:
            parsed = datetime.strptime(valor_str, self.FORMAT)
            self._data_hora = parsed
        except Exception as e:
            raise ValueError(f"Formato inválido para data_hora. Use '{self.FORMAT}'. Erro: {e}")

    def isPassado(self):
        if self._data_hora is None:
            return False
        return self._data_hora < datetime.now()

    def somaDias(self, num_dias: int):
        if self._data_hora is None:
            raise ValueError("data_hora não definida.")
        data_hora_somada = self._data_hora + timedelta(days=num_dias)
        return data_hora_somada.strftime(self.FORMAT)


### Teste da classe DataHora

In [ ]:

# instanciando o objeto
dh = DataHora()

# definindo a data_hora através da propriedade
dh.data_hora = '05/02/2024, 12:30'
print('Definido:', dh.data_hora)

# editando a data_hora através da função somaDias
dh.data_hora = dh.somaDias(30)
print('Após somaDias(30):', dh.data_hora, 'isPassado?', dh.isPassado())


## Classe EventoUnico

- Herda de `EventoABC`.
- Possui `_data_hora` do tipo `DataHora`.
- Construtor recebe título, descrição e data (string) e inicializa a propriedade interna do `DataHora`.
- Implementa `isConcluido()` e `__str__()`.
- Método `editar_data_hora()` para alterar a data.

In [ ]:

class EventoUnico(EventoABC):
    def __init__(self, titulo: str, descricao: str, data_hora_str: str):
        super().__init__(titulo, descricao)
        self._data_hora = DataHora()
        # usa a propriedade para validar/armazenar
        self._data_hora.data_hora = data_hora_str

    def isConcluido(self):
        return self._data_hora.isPassado()

    def editar_data_hora(self, nova_data_str: str):
        self._data_hora.data_hora = nova_data_str

    def __str__(self):
        return f"Evento: {self._titulo}, Data: {self._data_hora.data_hora}, Descrição: {self._descricao}, Concluido: {self.isConcluido()}"


### Teste da classe EventoUnico

In [ ]:

# criar evento
evento = EventoUnico('Reunião', 'Sala 302, prédio da esquina', '05/10/2023, 16:30')
print(evento)

# editar data do evento (através da propriedade)
evento.editar_data_hora('05/10/2024, 16:30')
print(evento)


## Classe EventoRecorrente

- Herda de `EventoABC`.
- Mantém uma lista privada de objetos `DataHora`.
- Construtor recebe data_hora_inicial, data_hora_final e intervalo_repeticao (dias) e preenche a coleção.
- Implementa `isConcluido(indice)` e `__str__()` que imprime todas as ocorrências.
- `editar_data_hora(data_hora_antiga, data_hora_nova)` altera o elemento correspondente.

In [ ]:

class EventoRecorrente(EventoABC):
    def __init__(self, titulo: str, descricao: str,
                 data_hora_inicial: str, data_hora_final: str, intervalo_repeticao: int):
        super().__init__(titulo, descricao)
        # lista privada de DataHora
        self._datas = []

        # criar DataHora inicial e final temporários para controle
        dh_inicial = DataHora()
        dh_inicial.data_hora = data_hora_inicial

        dh_final = DataHora()
        dh_final.data_hora = data_hora_final

        # adiciona ocorrências a cada intervalo_repeticao dias (inclusive a inicial; se final coincidir, inclui)
        atual = dh_inicial._data_hora  # datetime interno
        final_dt = dh_final._data_hora

        while atual <= final_dt:
            dh = DataHora()
            dh._data_hora = atual  # atribuir diretamente o datetime
            self._datas.append(dh)
            atual = atual + timedelta(days=intervalo_repeticao)

    def isConcluido(self, indice: int):
        if indice < 0 or indice >= len(self._datas):
            raise IndexError('Índice fora do intervalo de ocorrências.')
        return self._datas[indice].isPassado()

    def editar_data_hora(self, data_hora_antiga: str, data_hora_nova: str):
        # encontra o índice que corresponde à data_hora_antiga (string formatada)
        encontrado = False
        for i, dh in enumerate(self._datas):
            if dh.data_hora == data_hora_antiga:
                # atualiza usando a propriedade
                dh.data_hora = data_hora_nova
                encontrado = True
                break
        if not encontrado:
            raise ValueError('data_hora_antiga não encontrada nas ocorrências.')

    def __str__(self):
        linhas = []
        for i, dh in enumerate(self._datas):
            linhas.append(f"Evento: {self._titulo}, Data: {dh.data_hora}, Descrição: {self._descricao}, Concluido: {self.isConcluido(i)}")
        return "\n".join(linhas)


### Teste da classe EventoRecorrente

In [ ]:

# criar evento recorrente
eventos = EventoRecorrente(
    'Reunião', 'Sala 302, prédio da esquina',
    '05/01/2024, 16:30', '05/01/2025, 16:30', 30)

# imprimir eventos
print(eventos)

# editar um dos eventos
eventos.editar_data_hora('05/12/2024, 16:30', '05/12/2024, 11:30')

# imprimir eventos após edição
print('\n--- Após edição ---\n')
print(eventos)


## Polimorfismo: lista mista de eventos

Criamos uma lista com `EventoUnico` e `EventoRecorrente` e imprimimos todos usando `print()`, demonstrando polimorfismo.

In [ ]:

# Criando lista de eventos
lista_eventos = [
    EventoUnico('Apresentação', 'Auditório', '10/10/2024, 09:00'),
    eventos,  # EventoRecorrente criado antes
    EventoUnico('Entrevista', 'Sala RH', '01/11/2024, 14:00')
]

# imprimir todos os eventos (polimorfismo: cada __str__ é chamado conforme o tipo)
for evento in lista_eventos:
    print('\n--- Evento ---\n')
    print(evento)
